In [ ]:
from xgboost import XGBRegressor, XGBRFRegressor
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.ensemble import VotingRegressor
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pandas as pd
import tensorflow as tf

from imbd.trainers import ModelTrainer
from imbd.data import DataLoader
from imbd.preprocessors import DataPreprocessor
from imbd.models import KerasModel

In [ ]:
loader = DataLoader()
prepro = DataPreprocessor()


In [ ]:
# initialization
loader = DataLoader()
preprocessor = DataPreprocessor()
df = loader.build()

# get feature & label
train_features = df.drop(loader.labels, axis=1)
train_labels = df[loader.labels]

In [ ]:
order = [0, 2, 5, 7, 13, 14, 16, 17] + [1, 3, 4, 6, 8, 9, 11, 12, 15, 18, 19] + [10]
base_model = RegressorChain(LGBMRegressor(), order=order)

In [ ]:

param_grid = {
    # "prepro__variance_selector__threshold": [0.0],
    "base_estimator__boosting_type": ['gbdt'],
    # "base_estimator__tree_learner":
    # ["serial", "feature"],
    "base_estimator__max_depth": [2, 6, -1],
    "base_estimator__min_child_samples": [10, 20],
    "base_estimator__subsample": [0.5, 1],
    "base_estimator__n_estimators": [1000],
    # "base_estimator__num_leaves ": [15, 31],
    "base_estimator__colsample_bytree": [0.5, 1],
    "base_estimator__num_leaves": [4, 64],
    # "base_estimator__reg_alpha": [0.0, 0.05],
    # "base_estimator__reg_lambda": [0.0, 0.05],
}

In [ ]:
uniq_table = train_features.filter(regex='(Input_A[0-6]_[0-9]+|Output_A[0-6])').nunique().sort_values()
uniq_table[(uniq_table > 1) & (uniq_table < 10)].to_dict()

In [ ]:
# build pipeline
# steps = [('prepro', preprocessor), ('model', base_model)]
# pipe = Pipeline(steps=steps)
train_features = preprocessor.fit_transform(train_features)

In [ ]:
# training
trainer = ModelTrainer(base_model=base_model, param_grid=param_grid, verbose=2)
# fitted = trainer.fit(train_features, train_labels)
fitted = trainer.fit(train_features, train_labels)

In [ ]:
fitted.best_score_

In [ ]:
fitted.best_params_

In [ ]:
pd.DataFrame(fitted.cv_results_)

In [ ]:
test_loader = DataLoader(data_fp='data/0728test.csv', data_type='test')

In [ ]:
test_features = test_loader.build()
test_features = preprocessor.transform(test_features)
pred = fitted.predict(test_features)

In [ ]:
pd.DataFrame(pred, columns=loader.labels)